# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [8]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Glucocorticoid Sensitivity'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [9]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [10]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [11]:
trait_subdir = "TCGA_Bladder_Cancer_(BLCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BLCA.sampleMap_BLCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BLCA.sampleMap_HiSeqV2_PANCAN.gz')

In [12]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [13]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [14]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_BLCA',
 '_PANCAN_DNAMethyl_PANCAN',
 '_PANCAN_RPPA_PANCAN_K8',
 '_PANCAN_UNC_RNAseq_PANCAN_K16',
 '_PANCAN_miRNA_PANCAN',
 '_PANCAN_mirna_BLCA',
 '_PANCAN_mutation_PANCAN']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [15]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_BLCA', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_BLCA', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_treatment_completion_success_o

In [16]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [17]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [63, 66, 69, 59, 83],
 'days_to_birth': [-23323.0, -24428.0, -25259.0, -21848.0, -30520.0],
 'year_of_initial_pathologic_diagnosis': [2008.0,
  2011.0,
  2007.0,
  2000.0,
  2006.0]}

In [18]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [19]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'MALE']}

In [20]:
gender_col = 'gender'

In [21]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [22]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

13 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('RPL31P11', 2), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
10 input query terms found dup hits:	[('SUGT1P1', 2), ('PTPRVP', 2), ('SNORA62', 3), ('IFITM4P', 7), ('HLA-DRB6', 2), ('FUNDC2P2', 2), ('
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
11 input query terms found dup hits:	[('PIP5K1P1', 2), ('HBD', 2), ('PPP1R2P1', 9), ('HSD17B7P2', 2), ('RPSAP9', 2), ('SNORD68', 2), ('SN
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
15 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('ESPNP', 2), ('HBBP1
158 input query terms found no hit:	[

In [23]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Glucocorticoid Sensitivity,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO-01,1,63,1,0.221008,4.428374,-0.531035,0.720328,-0.408078,-0.39141,0.048706,...,-2.090786,-0.086682,-0.086078,-0.128017,0.094667,3.01571,0.049323,-0.418127,0.095905,1.301467
TCGA-2F-A9KP-01,1,66,1,1.334508,-2.541326,-0.531035,0.490728,0.523922,-0.59311,0.178806,...,-1.584586,-0.086682,0.124822,-1.085217,-2.360533,1.65561,-0.402877,-0.135527,0.417805,0.588167
TCGA-2F-A9KQ-01,1,69,1,1.173308,-5.554726,-0.531035,1.647728,0.518622,0.02829,0.800406,...,-1.541286,-0.086682,-0.748878,0.498683,-0.768233,-0.33489,-0.685077,0.392573,-0.115395,-1.106033
TCGA-2F-A9KR-01,1,59,0,1.090208,0.971574,-0.531035,0.256228,0.586622,-0.73021,0.636006,...,0.311914,-0.086682,0.214422,-1.587117,-1.082733,1.90011,0.422823,-1.583627,0.596105,-0.924533
TCGA-2F-A9KT-01,1,83,1,0.955008,-1.481926,0.115365,-0.039972,0.746322,-0.59151,0.365906,...,0.763114,-0.086682,-0.748878,-1.587117,-0.136933,1.87471,0.027523,-0.462527,-0.186195,-2.369433


In [24]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 426 samples.


In [25]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Glucocorticoid Sensitivity', the least common label is '0' with 19 occurrences. This represents 4.46% of the dataset.
The distribution of the feature 'Glucocorticoid Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 69.0
  75%: 76.0
Min: 34
Max: 90
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0' with 115 occurrences. This represents 27.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [26]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [27]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [6]:
import os
from utils import *

dataset = 'GEO'
trait_subdir = "Glucocorticoid-Sensitivity"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE38352',
 'GSE58715',
 'GSE32962',
 'GSE50012',
 'GSE15820',
 'GSE2842',
 'GSE65645',
 'GSE42002',
 'GSE57795',
 'GSE33649',
 'GSE5824',
 'GSE94302',
 'GSE67042',
 'GSE48801',
 'GSE67044',
 'GSE66705',
 'GSE66702']

Repeat the below steps for all the accession numbers

In [92]:
cohort = accession_num = "GSE66705"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Glucocorticoid-Sensitivity/GSE66705/GSE66705_family.soft.gz',
 '../../DATA/GEO/Glucocorticoid-Sensitivity/GSE66705/GSE66705_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [93]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"NALP3 inflammasome up-regulation and CASP1 cleavage of the glucocorticoid receptor causes glucocorticoid resistance in leukemia cells [HG-U133_Plus_2]"
!Series_summary	"Glucocorticoids are universally used in the treatment of acute lymphoblastic leukemia (ALL), and glucocorticoid resistance in leukemia cells confers a poor prognosis.  To elucidate mechanisms of glucocorticoid resistance, we determined the prednisolone sensitivity of primary leukemia cells from 444 newly diagnosed ALL patients and found significantly higher expression of caspase 1 (CASP1) and its activator NLRP3 in glucocorticoid resistant leukemia cells, due to significantly lower somatic methylation of CASP1 and NLRP3 promoters.  Over-expression of CASP1 resulted in cleavage of the glucocorticoid receptor, diminished glucocorticoid-induced transcriptional response and increased glucocorticoid resistance. Knockdown or inhibition of CASP1 significantly increased glucocorticoid receptor levels and mitigate

In [94]:
clinical_data.head()

,!Sample_geo_accession,GSM1629982,GSM1629983,GSM1629984,GSM1629985,GSM1629986,GSM1629987,GSM1629988,GSM1629989,GSM1629990,...,GSM1630156,GSM1630158,GSM1630160,GSM1630162,GSM1630163,GSM1630164,GSM1630165,GSM1630166,GSM1630167,GSM1630168
0,!Sample_characteristics_ch1,predlc50group: #N/A,predlc50group: #N/A,predlc50group: #N/A,predlc50group: #N/A,predlc50group: RES,predlc50group: #N/A,predlc50group: RES,predlc50group: RES,predlc50group: #N/A,...,predlc50group: SEN,predlc50group: INT,predlc50group: INT,predlc50group: SEN,predlc50group: RES,predlc50group: RES,predlc50group: RES,predlc50group: INT,predlc50group: SEN,predlc50group: INT
1,!Sample_characteristics_ch1,lin: B,lin: T,lin: B,lin: B,lin: B,lin: B,lin: B,lin: B,lin: B,...,lin: B,lin: B,lin: B,lin: B,lin: B,lin: B,lin: T,lin: B,lin: B,lin: B


In [95]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['predlc50group: #N/A',
  'predlc50group: RES',
  'predlc50group: SEN',
  'predlc50group: INT'],
 1: ['lin: B', 'lin: T']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [96]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Glucocorticoid Sensitivity\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Glucocorticoid Sensitivity\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [97]:
age_row = gender_row = None
convert_age = convert_gender = None

In [98]:
is_gene_availabe = True
trait_row = 0

trait_type = 'binary'
# trait_type = 'continuous'

In [101]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [102]:
def convert_trait(value):
    if value == 'predlc50group: SEN':
        return 1  # Sensitive
    elif value == 'predlc50group: RES':
        return 0  # Resistant
    else:
        return None  # Unknown or not applicable

In [103]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1629982,GSM1629983,GSM1629984,GSM1629985,GSM1629986,GSM1629987,GSM1629988,GSM1629989,GSM1629990,GSM1629991,...,GSM1630156,GSM1630158,GSM1630160,GSM1630162,GSM1630163,GSM1630164,GSM1630165,GSM1630166,GSM1630167,GSM1630168
Glucocorticoid Sensitivity,None,None,None,None,0,None,0,0,None,None,...,1,None,None,1,0,0,0,None,1,None


### Genetic data preprocessing and final filtering

In [104]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1629982,GSM1629983,GSM1629984,GSM1629985,GSM1629986,GSM1629987,GSM1629988,GSM1629989,GSM1629990,GSM1629991,...,GSM1630156,GSM1630158,GSM1630160,GSM1630162,GSM1630163,GSM1630164,GSM1630165,GSM1630166,GSM1630167,GSM1630168
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,10.130815,9.608832,9.859652,10.587496,8.271685,8.955985,10.137842,10.246692,9.160129,9.533762,...,10.529306,10.475289,10.202895,10.427700,10.135887,9.810805,9.379958,9.394382,10.024207,8.810702
1053_at,9.553469,9.898287,9.713977,10.219711,9.607441,9.655327,9.987416,10.669453,9.967548,9.835980,...,9.621368,9.598187,10.025506,9.694897,10.098302,9.914559,9.775615,10.024191,9.713482,9.581438
117_at,7.001835,6.133293,6.751620,5.757822,9.184924,8.186819,8.441796,6.002942,6.980694,5.574980,...,5.603993,6.121035,6.056828,6.971651,6.398391,5.606276,8.228963,7.328117,6.054414,5.171814
121_at,9.042864,8.624124,8.489150,8.201862,8.816792,9.000001,8.462935,8.917688,8.965709,8.114824,...,8.445530,8.494710,7.837700,8.025939,8.160592,8.082318,8.430132,8.247690,8.451463,8.742645
1255_g_at,5.830282,4.769006,5.169443,5.392124,5.176441,5.518506,5.130876,3.179718,6.050559,2.864493,...,5.135100,3.018319,2.330385,4.090487,3.354563,2.404750,2.643028,2.852257,5.432093,2.916786


In [105]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1552256_a_at',
 '1552257_a_at',
 '1552258_at',
 '1552261_at',
 '1552263_at',
 '1552264_a_at',
 '1552266_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [106]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']\n"

If not required, jump directly to the gene normalization step

In [107]:
requires_gene_mapping = True

In [108]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [109]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_name_key = key_name1
gene_name_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

'\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_name_key = key_name1\ngene_name_key = key_name2\n\nGene annotation dictionary:\n{\'ID\': [\'1007_s_at\', \'1053_at\', \'117_at\', \'121_at\', \'1255_g_at\'], \'GB_ACC\': [\'U48705\', \'M87338\', \'X51757\', \'X69699\', \'L36861\'], \'SPOT_ID\': [nan, nan, nan, nan, nan], \'Species Scientific Name\': [\'Homo sapiens\', \'Homo sapiens\', \'Homo sapiens\', \'Homo sapiens\', \'Homo sapiens\'], \'Annotation Date\': [\'Oct 6, 2014\', \'Oct 6, 2014\', \'Oct 6, 2014\', \'Oct 6, 2014\', \'Oct 6, 2014\'], \'Sequence Type\': [\'Exemplar sequence\', \'Exemplar sequence\', \'Exemplar sequence\', \'Exemplar sequence\', \'Exemplar sequence\'], \'Sequence Source\': [\'Affymetrix Proprietary 

In [110]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [111]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

200 input query terms found dup hits:	[('ABCC6P1', 2), ('ABCC6P2', 3), ('ABCD1P2', 3), ('ACTBP9', 2), ('ACTR3BP5', 3), ('ACTR3P2', 2), ('A
409 input query terms found no hit:	['AC078883.4', 'AF279782', 'AK023507', 'AK055364', 'AK055967', 'AK056973', 'AK094915', 'AK096803', '
182 input query terms found dup hits:	[('LONP2', 2), ('LRRC37A16P', 2), ('LRRC37A2', 2), ('LRRC37A4P', 3), ('LRRC37A5P', 2), ('LRRC37A7P',
206 input query terms found no hit:	['LOC285389', 'LOC285835', 'LOC388572', 'LOC388692', 'LOC399491', 'LOC400927', 'LOC401317', 'LOC4013
137 input query terms found dup hits:	[('SNORD19B', 2), ('SNORD1A', 2), ('SNORD1C', 2), ('SNORD2', 2), ('SNORD29', 2), ('SNORD33', 2), ('S
186 input query terms found no hit:	['SOGA3', 'SOX5P', 'SPHAR', 'SSU72P8', 'TCEB1P28', 'TCEB1P3', 'TCEB3C', 'TCEB3CL', 'TCEB3CL2', 'TCR-
59 input query terms found dup hits:	[('AGAP11', 4), ('AGAP6', 2), ('AGO2', 2), ('AHCTF1', 2), ('AK6', 2), ('AKIP1', 2), ('AKR1C1', 2), (
155 input query terms found no hit

In [112]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [113]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 84 samples.


In [114]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Glucocorticoid Sensitivity', the least common label is '0.0' with 32 occurrences. This represents 38.10% of the dataset.
The distribution of the feature 'Glucocorticoid Sensitivity' in this dataset is fine.



False

In [115]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [116]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [28]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
14,Xena,True,True,False,True,True,426.0,
10,GSE48801,True,True,False,False,False,179.0,
13,GSE66702,True,True,False,False,False,142.0,
12,GSE66705,True,True,False,False,False,84.0,
2,GSE32962,True,True,False,False,False,43.0,


In [29]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'Xena'

In [30]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
14,Xena,True,True,False,True,True,426.0,


In [33]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Glucocorticoid Sensitivity,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,63,1,0.221008,4.428374,-0.531035,0.720328,-0.408078,-0.39141,0.048706,...,-2.090786,-0.086682,-0.086078,-0.128017,0.094667,3.01571,0.049323,-0.418127,0.095905,1.301467
1,1,66,1,1.334508,-2.541326,-0.531035,0.490728,0.523922,-0.59311,0.178806,...,-1.584586,-0.086682,0.124822,-1.085217,-2.360533,1.65561,-0.402877,-0.135527,0.417805,0.588167
2,1,69,1,1.173308,-5.554726,-0.531035,1.647728,0.518622,0.02829,0.800406,...,-1.541286,-0.086682,-0.748878,0.498683,-0.768233,-0.33489,-0.685077,0.392573,-0.115395,-1.106033
3,1,59,0,1.090208,0.971574,-0.531035,0.256228,0.586622,-0.73021,0.636006,...,0.311914,-0.086682,0.214422,-1.587117,-1.082733,1.90011,0.422823,-1.583627,0.596105,-0.924533
4,1,83,1,0.955008,-1.481926,0.115365,-0.039972,0.746322,-0.59151,0.365906,...,0.763114,-0.086682,-0.748878,-1.587117,-0.136933,1.87471,0.027523,-0.462527,-0.186195,-2.369433


In [34]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [35]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [36]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [37]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 64.47% ± 5.92%


In [38]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 76.94% ± 4.80%


In [39]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 4. Discussion and report

In [40]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0073
Found 17 genes with non-zero coefficients associated with the trait 'Glucocorticoid Sensitivity' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
   SMR3A    -0.957890              0.957890
  NKX6-2     0.814918              0.814918
    FGF6    -0.618771              0.618771
 TBC1D3G     0.491900              0.491900
   IL17F    -0.481053              0.481053
 TMEM213    -0.471884              0.471884
  RSPH6A     0.425530              0.425530
  OR2A25     0.372596              0.372596
 TBC1D3C    -0.318377              0.318377
   KCTD4     0.287430              0.287430
    LGSN     0.160146              0.160146
 TAS2R30     0.129086              0.129086
    PIM3     0.120899              0.120899
  OR2A14     0.116630              0.116630
   DCTN5     0.109493              0.109493
  CLDN10  

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [41]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0073
Found 41 significant genes associated with the trait 'Glucocorticoid Sensitivity' conditional on the factor 'Age', with corrected p-value < 0.05:
 Variable  Coefficient      p_value  corrected_p_value
    MMP27    -0.066884 2.952067e-19       5.118294e-15
   CLEC3B    -0.086405 3.629278e-09       3.146221e-05
    LMX1A    -0.049278 1.218598e-08       7.042685e-05
  COL10A1     0.083467 2.016525e-08       8.740628e-05
  FAM180B    -0.052842 3.245674e-08       1.125470e-04
    TAAR6    -0.037217 4.530768e-08       1.309241e-04
   OLFML1    -0.097990 7.751943e-08       1.920045e-04
   VSTM2A    -0.045337 1.271020e-07       2.754619e-04
     ESM1     0.060856 1.591626e-07       3.066180e-04
     CST2     0.042600 2.869742e-07       4.975558e-04
   OR2T27    -0.035234 9.670513e-07       1.524249e-03
    INHBA     0.064166 1.359063e-06       1.942670e-03
   COL4A1     0.069119 1.456610e-06       1.942670e-03
 

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
